In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

     |████████████████████████████████| 274 kB 4.1 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=2f2f9647b118ff06d1c582b91c8d714d38d6ca7b5352769726407d7559486bc0
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
torch:  1.10 ; cuda:  cu111
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html
     |████████████████████████████████| 7.0 MB 1.4 MB/s 
     |████████████████████████████████| 130 kB 6.3 MB/s 
     |████████████████████████████████| 145 kB 47.3 MB/s 
     |████████████████████████████████| 49 kB 9.6 MB/s 
     |████████████████████████████████| 74 kB 4.7 MB/s 
     |████████████████████████████████| 843 kB 72.3 MB/s 
     |████████████████████████████████

In [ ]:
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git


In [1]:
datapath = '/content/drive/MyDrive/PyPSA_Africa_images/datasets'

import sys
sys.argv = ['']

In [2]:
# Add pypsa africa tool
import os
os.chdir('/content/drive/MyDrive/PyPSA_Africa_images/unbiased-teacher/')

import sys
sys.path.append('/content/drive/MyDrive/PyPSA_Africa_images/detect_energy/src/')

In [3]:
import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.engine import default_argument_parser, default_setup, launch

from ubteacher import add_ubteacher_config
from ubteacher.engine.trainer import UBTeacherTrainer, BaselineTrainer

# hacky way to register
from ubteacher.modeling.meta_arch.rcnn import TwoStagePseudoLabGeneralizedRCNN
from ubteacher.modeling.proposal_generator.rpn import PseudoLabRPN
from ubteacher.modeling.roi_heads.roi_heads import StandardROIHeadsPseudoLab
from ubteacher.modeling.meta_arch.rcnn import TwoStagePseudoLabGeneralizedRCNN
import ubteacher.data.datasets.builtin

from ubteacher.modeling.meta_arch.ts_ensemble import EnsembleTSModel

# import some of our own methods
from utils.detectron_utils import eval_predictor
from utils.image_utils import get_true_images

In [4]:
def main(args):
    cfg = setup(args)

    if cfg.SEMISUPNET.Trainer == "ubteacher":
        Trainer = UBTeacherTrainer
    elif cfg.SEMISUPNET.Trainer == "baseline":
        Trainer = BaselineTrainer
    else:
        raise ValueError("Trainer Name is not found.")

    trainer = Trainer(cfg)    
    trainer.train()
    
    return None

In [11]:
from detectron2 import model_zoo

def setup(args):

    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    # print(cfg)
    add_ubteacher_config(cfg)
    # print(cfg)
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    # cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml'))
    cfg.freeze()
    default_setup(cfg, args)
    return cfg


In [12]:
import os
import time
import logging
import torch
from torch.nn.parallel import DistributedDataParallel
from fvcore.nn.precise_bn import get_bn_modules
import numpy as np
from collections import OrderedDict

import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.engine import DefaultTrainer, SimpleTrainer, TrainerBase
from detectron2.engine.train_loop import AMPTrainer
from detectron2.utils.events import EventStorage
from detectron2.evaluation import COCOEvaluator, verify_results, PascalVOCDetectionEvaluator, DatasetEvaluators
from detectron2.data.dataset_mapper import DatasetMapper
from detectron2.engine import hooks
from detectron2.structures.boxes import Boxes
from detectron2.structures.instances import Instances
from detectron2.utils.env import TORCH_VERSION
from detectron2.data import MetadataCatalog
from detectron2.modeling import build_model
from detectron2 import model_zoo

from ubteacher.data.build import (
    build_detection_semisup_train_loader,
    build_detection_test_loader,
    build_detection_semisup_train_loader_two_crops,
)
from ubteacher.data.dataset_mapper import DatasetMapperTwoCropSeparate
from ubteacher.engine.hooks import LossEvalHook
from ubteacher.modeling.meta_arch.ts_ensemble import EnsembleTSModel
from ubteacher.checkpoint.detection_checkpoint import DetectionTSCheckpointer
from ubteacher.solver.build import build_lr_scheduler

In [88]:
import numpy as np
import logging
import operator

from detectron2.modeling import build_model
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.data.build import get_detection_dataset_dicts, build_batch_data_loader
from detectron2.data.common import DatasetFromList, MapDataset
from detectron2.data.samplers import TrainingSampler
from detectron2.data.dataset_mapper import DatasetMapper
from detectron2.utils.comm import get_world_size
from detectron2.data.build import (
    trivial_batch_collator,
    worker_init_reset_seed,
    get_detection_dataset_dicts,
    build_batch_data_loader,
)


from ubteacher.data.build import divide_label_unlabel
from ubteacher.data.common import (
    AspectRatioGroupedSemiSupDatasetTwoCrop,
)
from ubteacher.modeling.meta_arch.ts_ensemble import EnsembleTSModel

from itertools import product


# batch data loader
def build_semisup_batch_data_loader_two_crop(
    dataset,
    sampler,
    total_batch_size_label,
    total_batch_size_unlabel,
    *,
    aspect_ratio_grouping=False,
    num_workers=0
):
    world_size = get_world_size()
    assert (
        total_batch_size_label > 0 and total_batch_size_label % world_size == 0
    ), "Total label batch size ({}) must be divisible by the number of gpus ({}).".format(
        total_batch_size_label, world_size
    )

    assert (
        total_batch_size_unlabel > 0 and total_batch_size_unlabel % world_size == 0
    ), "Total unlabel batch size ({}) must be divisible by the number of gpus ({}).".format(
        total_batch_size_label, world_size
    )

    batch_size_label = total_batch_size_label // world_size
    batch_size_unlabel = total_batch_size_unlabel // world_size

    label_dataset, unlabel_dataset = dataset
    label_sampler, unlabel_sampler = sampler

    if aspect_ratio_grouping:
        label_data_loader = torch.utils.data.DataLoader(
            label_dataset,
            sampler=label_sampler,
            num_workers=num_workers,
            batch_sampler=None,
            collate_fn=operator.itemgetter(
                0
            ),  # don't batch, but yield individual elements
            worker_init_fn=worker_init_reset_seed,
        )  # yield individual mapped dict
        unlabel_data_loader = torch.utils.data.DataLoader(
            unlabel_dataset,
            sampler=unlabel_sampler,
            num_workers=num_workers,
            batch_sampler=None,
            collate_fn=operator.itemgetter(
                0
            ),  # don't batch, but yield individual elements
            worker_init_fn=worker_init_reset_seed,
        )  # yield individual mapped dict
        return AspectRatioGroupedSemiSupDatasetTwoCrop(
            (label_data_loader, unlabel_data_loader),
            (batch_size_label, batch_size_unlabel),
        )
    else:
        raise NotImplementedError("ASPECT_RATIO_GROUPING = False is not supported yet")


def build_maxar_loader_semisup_two_crops(cfg, mapper=None):
    '''
    Sets up loader for custom maxar data
    '''

    # register used datasets
    ds_names = ['fake_maxar', 'maxar']
    modes = ['train', 'val']

    for name, mode in product(ds_names, modes):

        ds_name = f'{name}_{mode}'
        json_path = f'/content/drive/My Drive/PyPSA_Africa_images/datasets/{ds_name}/labels.json'
        ds_path = f'/content/drive/My Drive/PyPSA_Africa_images/datasets/{ds_name}/data/' 
    
        if ds_name in DatasetCatalog.list():
            DatasetCatalog.remove(ds_name)
            MetadataCatalog.remove(ds_name)
    
        register_coco_instances(ds_name, {}, json_path, ds_path)
    
    unlabel_dicts = get_detection_dataset_dicts(
                            ['maxar_train', 'maxar_val'],
                            filter_empty=False,
                            min_keypoints=0,
                            proposal_files=None
                        )

    label_dicts = get_detection_dataset_dicts(
                            ['fake_maxar_train', 'fake_maxar_val'],
                            filter_empty=False,
                            min_keypoints=0,
                            proposal_files=None
                        )
 
    # cfg.DATASETS.SUP_PERCENT = int(100 * len(label_dicts) / (len(label_dicts) + len(unlabel_dicts)))
     
    label_dataset = DatasetFromList(label_dicts, copy=False)
    unlabel_dataset = DatasetFromList(unlabel_dicts, copy=False)

    if mapper is None:
        mapper = DatasetMapper(cfg, True)
    label_dataset = MapDataset(label_dataset, mapper)
    unlabel_dataset = MapDataset(unlabel_dataset, mapper)

    sampler_name = cfg.DATALOADER.SAMPLER_TRAIN
    logger = logging.getLogger(__name__)
    logger.info("Using training sampler {}".format(sampler_name))
    if sampler_name == "TrainingSampler":
        label_sampler = TrainingSampler(len(label_dataset))
        unlabel_sampler = TrainingSampler(len(unlabel_dataset))
    elif sampler_name == "RepeatFactorTrainingSampler":
        raise NotImplementedError("{} not yet supported.".format(sampler_name))
    else:
        raise ValueError("Unknown training sampler: {}".format(sampler_name))
    return build_semisup_batch_data_loader_two_crop(
        (label_dataset, unlabel_dataset),
        (label_sampler, unlabel_sampler),
        cfg.SOLVER.IMG_PER_BATCH_LABEL,
        cfg.SOLVER.IMG_PER_BATCH_UNLABEL,
        aspect_ratio_grouping=cfg.DATALOADER.ASPECT_RATIO_GROUPING,
        num_workers=cfg.DATALOADER.NUM_WORKERS,
    )

# cfg.DATASETS.TRAIN = ['maxar_train', 'maxar_val']




In [89]:
import logging
import torchvision.transforms as transforms
from ubteacher.data.transforms.augmentation_impl import GaussianBlur

def build_strong_maxar_augmentation(cfg, is_train):

    logger = logging.getLogger(__name__)
    augmentation = []

    if is_train:
        # This is simialr to SimCLR https://arxiv.org/abs/2002.05709
        augmentation.append(
            transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8)
        )
        augmentation.append(transforms.RandomGrayscale(p=0.2))
        augmentation.append(transforms.RandomApply([GaussianBlur([0.1, 2.0])], p=0.5))

        other_transform = transforms.Compose(
            [
                transforms.ToTensor(),
                # transforms.RandomErasing(
                #     p=0.7, scale=(0.05, 0.2), ratio=(0.3, 3.3), value="random"
                # ),
                # transforms.RandomErasing(
                #     p=0.5, scale=(0.02, 0.2), ratio=(0.1, 6), value="random"
                # ),
                # transforms.RandomErasing(
                #     p=0.3, scale=(0.02, 0.2), ratio=(0.05, 8), value="random"
                # ),
                transforms.ToPILImage(),
            ]
        )
        augmentation.append(other_transform)

        logger.info("Augmentations used in training: " + str(augmentation))
    return transforms.Compose(augmentation)


In [90]:
from detectron2.data.dataset_mapper import DatasetMapper
import detectron2.data.detection_utils as utils

from ubteacher.data.dataset_mapper import DatasetMapperTwoCropSeparate


class MaxarDatasetMapper(DatasetMapperTwoCropSeparate):
    def __init__(self, cfg, is_train=True):
        self.augmentation = utils.build_augmentation(cfg, is_train)
        self.compute_tight_boxes = False
        self.strong_augmentation = build_strong_maxar_augmentation(cfg, is_train)

        # fmt: off
        self.img_format = cfg.INPUT.FORMAT
        self.mask_on = cfg.MODEL.MASK_ON
        self.mask_format = cfg.INPUT.MASK_FORMAT
        self.keypoint_on = cfg.MODEL.KEYPOINT_ON
        self.load_proposals = cfg.MODEL.LOAD_PROPOSALS
        # fmt: on
        if self.keypoint_on and is_train:
            self.keypoint_hflip_indices = utils.create_keypoint_hflip_indices(
                cfg.DATASETS.TRAIN
            )
        else:
            self.keypoint_hflip_indices = None

        if self.load_proposals:
            self.proposal_min_box_size = cfg.MODEL.PROPOSAL_GENERATOR.MIN_SIZE
            self.proposal_topk = (
                cfg.DATASETS.PRECOMPUTED_PROPOSAL_TOPK_TRAIN
                if is_train
                else cfg.DATASETS.PRECOMPUTED_PROPOSAL_TOPK_TEST
            )
        self.is_train = is_train



In [91]:
from detectron2.modeling.meta_arch.build import META_ARCH_REGISTRY
from detectron2.modeling.meta_arch.rcnn import GeneralizedRCNN
import pprint

if 'MaxarPseudoLabGeneralizedRCNN' in META_ARCH_REGISTRY:
    META_ARCH_REGISTRY.__dict__['_obj_map'].pop('MaxarPseudoLabGeneralizedRCNN')

@META_ARCH_REGISTRY.register()
class MaxarPseudoLabGeneralizedRCNN(GeneralizedRCNN):

    def forward(
        self, batched_inputs, branch="supervised", given_proposals=None, val_mode=False
    ):
        if (not self.training) and (not val_mode) and (not branch=='unsup_data_weak'):
            print('actually we are doing this')
            return self.inference(batched_inputs)

        # print('input received in the forward')
        # pprint.pprint(batched_inputs)

        images = self.preprocess_image(batched_inputs)

        if "instances" in batched_inputs[0]:
            gt_instances = [x["instances"].to(self.device) for x in batched_inputs]
        else:
            gt_instances = None

        features = self.backbone(images.tensor)

        if branch == "supervised":
            # Region proposal network
            proposals_rpn, proposal_losses = self.proposal_generator(
                images, features, gt_instances
            )

            # # roi_head lower branch
            _, detector_losses = self.roi_heads(
                images, features, proposals_rpn, gt_instances, branch=branch
            )

            losses = {}
            losses.update(detector_losses)
            losses.update(proposal_losses)
            return losses, [], [], None

        elif branch == "unsup_data_weak":

            # Region proposal network
            # print('type proposal generator')
            # print(type(self.proposal_generator))
            proposals_rpn, _ = self.proposal_generator(
                images, features, None, compute_loss=False
            )

            # roi_head lower branch (keep this for further production)  
            # notice that we do not use any target in ROI head to do inference !

            # if self.roi_heads.training:
            #    kwargs = {'targets': None, 'compute_loss': False, 'branch': branch} 
            # else:
            #    kwargs = {'targets': None, 'branch': branch} 

            proposals_roih, ROI_predictions = self.roi_heads(
                images,
                features,
                proposals_rpn,
                targets=None,
                compute_loss=False,
                branch=branch,
            )

            # print('Here still all is good?')
            return {}, proposals_rpn, proposals_roih, ROI_predictions

        elif branch == "val_loss":
 
            # Region proposal network
            proposals_rpn, proposal_losses = self.proposal_generator(
                images, features, gt_instances, compute_val_loss=True
            )

            # roi_head lower branch
            _, detector_losses = self.roi_heads(
                images,
                features,
                proposals_rpn,
                gt_instances,
                branch=branch,
                compute_val_loss=True,
            )

            losses = {}
            losses.update(detector_losses)
            losses.update(proposal_losses)
            return losses, [], [], None

In [92]:
from google.colab.patches import cv2_imshow

from detectron2.utils.visualizer import Visualizer
from detectron2.structures import BoxMode
from detectron2.data import detection_utils as utils
import matplotlib.pyplot as plt
from scipy.special import softmax
from scipy.special import expit
plt.style.use('bmh')


def show_data(datalist, metadata=MetadataCatalog.get('fake_maxar_val'), final_output=True):

    if final_output:
        try:
            datalist = change_classes(datalist)
        except KeyError:
            pass
    
    for data in datalist:
        img = data["image"].permute(1, 2, 0).cpu().detach().numpy()
        img = utils.convert_image_to_rgb(img, cfg.INPUT.FORMAT)        

        visualizer = Visualizer(img, metadata=metadata, scale=0.6)

        if 'instances' in data:
            target_fields = data["instances"].get_fields()

            if 'proposal_boxes' in target_fields:

                objectness = [expit(val.cpu()) for val in target_fields['objectness_logits']]

                for prob, (i, box) in zip(objectness, enumerate(target_fields['proposal_boxes'].to('cpu'))):

                    if prob < max(objectness):
                        continue

                    visualizer.draw_box(box)
                    visualizer.draw_text(prob.item(), tuple(box[:2].numpy()))

                cv2_imshow(visualizer.get_output().get_image()[:,:,::-1])
                return

            elif 'gt_classes' in target_fields:
                labels = [metadata.thing_classes[i] for i in target_fields["gt_classes"]]
                visualizer = visualizer.overlay_instances(
                    labels=labels,
                    boxes=target_fields.get("gt_boxes", None),
                    masks=target_fields.get("gt_masks", None),
                    keypoints=target_fields.get("gt_keypoints", None),
                )
            else:
                visualizer = visualizer.draw_instance_predictions(data['instances'])

            cv2_imshow(visualizer.get_image()[:,:,::-1])
        
        else:
            cv2_imshow(visualizer.get_output().get_image()[:,:,::-1])


def filter_instances(data):
    '''
    Removes all instances in data that are faulty.
    E.g. have no extent

    Args:
        data(List[Dict]): each list entry is one image
    '''

    for i, datum in enumerate(data):
        
        try:
            insts = datum['instances']
        except KeyError:
            continue

        # filter instances that have area zero
        t = insts.gt_boxes.tensor.cpu()
        num = len(insts)
        mask = (t[:,0] - t[:,2]) != torch.zeros(num)
        mask *= (t[:,1] - t[:,3]) != torch.zeros(num)
        insts = insts[mask]

        datum['instances'] = insts

    return data


In [109]:
from ubteacher.modeling.meta_arch.rcnn import TwoStagePseudoLabGeneralizedRCNN
from detectron2.utils.visualizer import Visualizer
from detectron2.structures import BoxMode
import matplotlib.pyplot as plt
plt.style.use('bmh')
import numpy as np

def change_classes(data):
    for i, datum in enumerate(data):
        # print(datum['instances'])
        classes = getattr(datum['instances'], 'gt_classes')
        setattr(datum['instances'], 'gt_classes', torch.ones_like(classes, dtype=int))
        data[i] = datum
    
    return data

class MaxarUBTeacher(DefaultTrainer):
    '''
    Unbiased teacher adapted to the transfer of training performance from 
    duke to maxar images

    This currently completely omits the BURN-IN stage and presumes it a 
    as already complete
    '''

    def __init__(self, cfg):
        cfg = DefaultTrainer.auto_scale_workers(cfg, comm.get_world_size())

        model = self.build_model(cfg)
        self.model_teacher = self.build_model(cfg)

        data_loader = self.build_train_loader(cfg)
        optimizer = self.build_optimizer(cfg, model)

        if comm.get_world_size() > 1:
            model = DistributedDataParallel(
                model, device_ids=[comm.get_local_rank()], broadcast_buffers=False
            )

        TrainerBase.__init__(self)
        self._trainer = SimpleTrainer(model, data_loader, optimizer)

        ensem_ts_model = EnsembleTSModel(self.model_teacher, model)

        self.scheduler = self.build_lr_scheduler(cfg, optimizer)
        self.checkpointer = DetectionCheckpointer(
            ensem_ts_model,
            cfg.OUTPUT_DIR,
            optimizer=optimizer,
            scheduler=self.scheduler,
        )
        self.start_iter = 0
        self.max_iter = cfg.SOLVER.MAX_ITER
        self.cfg = cfg

        self.register_hooks(self.build_hooks())
        print('setup ub teacher complete!')

    # =====================================================
    # ================== Pseduo-labeling ==================
    # =====================================================

    def threshold_bbox(self, proposal_bbox_inst, thres=0.7, proposal_type="roih"):
        if proposal_type == "rpn":
            valid_map = proposal_bbox_inst.objectness_logits > thres

            # create instances containing boxes and gt_classes
            image_shape = proposal_bbox_inst.image_size
            new_proposal_inst = Instances(image_shape)

            # create box
            new_bbox_loc = proposal_bbox_inst.proposal_boxes.tensor[valid_map, :]
            new_boxes = Boxes(new_bbox_loc)

            # add boxes to instances
            new_proposal_inst.gt_boxes = new_boxes
            new_proposal_inst.objectness_logits = proposal_bbox_inst.objectness_logits[
                valid_map
            ]
        elif proposal_type == "roih":
            valid_map = proposal_bbox_inst.scores > thres

            # create instances containing boxes and gt_classes
            image_shape = proposal_bbox_inst.image_size
            new_proposal_inst = Instances(image_shape)

            # create box
            new_bbox_loc = proposal_bbox_inst.pred_boxes.tensor[valid_map, :]
            new_boxes = Boxes(new_bbox_loc)

            # add boxes to instances
            new_proposal_inst.gt_boxes = new_boxes
            new_proposal_inst.gt_classes = proposal_bbox_inst.pred_classes[valid_map]
            new_proposal_inst.scores = proposal_bbox_inst.scores[valid_map]

        return new_proposal_inst

    def process_pseudo_label(
        self, proposals_rpn_unsup_k, cur_threshold, proposal_type, psedo_label_method=""
    ):
        list_instances = []
        num_proposal_output = 0.0
        for proposal_bbox_inst in proposals_rpn_unsup_k:
            # thresholding
            if psedo_label_method == "thresholding":
                proposal_bbox_inst = self.threshold_bbox(
                    proposal_bbox_inst, thres=cur_threshold, proposal_type=proposal_type
                )
            else:
                raise ValueError("Unkown pseudo label boxes methods")
            num_proposal_output += len(proposal_bbox_inst)
            list_instances.append(proposal_bbox_inst)
        num_proposal_output = num_proposal_output / len(proposals_rpn_unsup_k)
        return list_instances, num_proposal_output

    def remove_label(self, label_data):
        for label_datum in label_data:
            if "instances" in label_datum.keys():
                del label_datum["instances"]
        return label_data

    def add_label(self, unlabled_data, label):
        for unlabel_datum, lab_inst in zip(unlabled_data, label):
            unlabel_datum["instances"] = lab_inst
        return unlabled_data

    # =====================================================
    # =================== Training Flow ===================
    # =====================================================
    
    def train(self):
        self.train_loop(self.start_iter, self.max_iter)
        if hasattr(self, "_last_eval_results") and comm.is_main_process():
            verify_results(self.cfg, self._last_eval_results)
            return self._last_eval_results


    def train_loop(self, start_iter: int, max_iter: int):
        logger = logging.getLogger(__name__)
        logger.info('Starting from iteration {}'.format(start_iter))

        self.iter = self.start_iter = start_iter
        self.max_iter = max_iter

        with EventStorage(start_iter) as self.storage:
            try:
                self.before_train()

                for self.iter in range(start_iter, max_iter):

                    self.before_step()
                    self.run_step_full_semisup()
                    self.after_step()

            except Exception:
                logger.exception("Exception during training:")
                raise
            finally:
                self.after_train()

    @torch.no_grad()
    def _update_teacher_model(self, keep_rate=0.996):
        if comm.get_world_size() > 1:
            student_model_dict = {
                key[7:]: value for key, value in self.model.state_dict().items()
            }
        else:
            student_model_dict = self.model.state_dict()

        new_teacher_dict = OrderedDict()
        for key, value in self.model_teacher.state_dict().items():
            if key in student_model_dict.keys():
                new_teacher_dict[key] = (
                    student_model_dict[key] *
                    (1 - keep_rate) + value * keep_rate
                )
            else:
                raise Exception("{} is not found in student model".format(key))

        self.model_teacher.load_state_dict(new_teacher_dict)

    def _write_metrics(self, metrics_dict: dict):
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }

        # gather metrics among all workers for logging
        # This assumes we do DDP-style training, which is currently the only
        # supported method in detectron2.
        all_metrics_dict = comm.gather(metrics_dict)
        # all_hg_dict = comm.gather(hg_dict)

        if comm.is_main_process():
            if "data_time" in all_metrics_dict[0]:
                # data_time among workers can have high variance. The actual latency
                # caused by data_time is the maximum among workers.
                data_time = np.max([x.pop("data_time")
                                   for x in all_metrics_dict])
                self.storage.put_scalar("data_time", data_time)

            # average the rest metrics
            metrics_dict = {
                k: np.mean([x[k] for x in all_metrics_dict])
                for k in all_metrics_dict[0].keys()
            }

            # append the list
            loss_dict = {}
            for key in metrics_dict.keys():
                if key[:4] == "loss":
                    loss_dict[key] = metrics_dict[key]

            total_losses_reduced = sum(loss for loss in loss_dict.values())

            self.storage.put_scalar("total_loss", total_losses_reduced)
            if len(metrics_dict) > 1:
                self.storage.put_scalars(**metrics_dict)


    def run_step_full_semisup(self):
        self._trainer.iter = self.iter
        assert self.model.training, 'self.model was changed to eval mode!'

        start = time.perf_counter()
        data = next(self._trainer._data_loader_iter)
        label_data_q, label_data_k, unlabel_data_q, unlabel_data_k = data
        data_time = time.perf_counter() - start
        # print('unlabel_data_k')
        # show_data(unlabel_data_k)

        # remove unlabeled data labels
        unlabel_data_q = self.remove_label(unlabel_data_q)
        unlabel_data_k = self.remove_label(unlabel_data_k)


        if self.iter % self.cfg.SEMISUPNET.TEACHER_UPDATE_ITER == 0:
            self._update_teacher_model(keep_rate=self.cfg.SEMISUPNET.EMA_KEEP_RATE) 

        record_dict = {}





        # Get everything at pseudo-labelling
        global student
        global teacher 
        global loader 
        global checkpointer

        student = self.model 
        teacher = self.model_teacher
        loader = self._trainer._data_loader_iter
        checkpointer = self.checkpointer











        with torch.no_grad():
            try:
                # print('what we are putting inside the fct')
                # print(unlabel_data_k)
                # print(type(unlabel_data_k))
                (
                _,
                proposals_rpn_unsup_k,
                proposals_roih_unsup_k,
                _,
                ) = self.model_teacher(unlabel_data_k, branch="unsup_data_weak")
            except FloatingPointError:
                import pprint
                print('unlabel data k')
                pprint.pprint(unlabel_data_k)
                dummy = jfoisndvoc



        cur_threshold = self.cfg.SEMISUPNET.BBOX_THRESHOLD

        joint_proposal_dict = {}
        joint_proposal_dict['proposals_rpn'] = proposals_rpn_unsup_k

        (
            pesudo_proposals_rpn_unsup_k,
            nun_pseudo_bbox_rpn,
        ) = self.process_pseudo_label(
            proposals_rpn_unsup_k, cur_threshold, "rpn", "thresholding"
        )

        joint_proposal_dict["proposals_pseudo_rpn"] = pesudo_proposals_rpn_unsup_k
        # Pseudo_labeling for ROI head (bbox location/objectness)
        pesudo_proposals_roih_unsup_k, _ = self.process_pseudo_label(
            proposals_roih_unsup_k, cur_threshold, "roih", "thresholding"
        )
        joint_proposal_dict["proposals_pseudo_roih"] = pesudo_proposals_roih_unsup_k   


        unlabel_data_q = self.add_label(
            unlabel_data_q, joint_proposal_dict["proposals_pseudo_roih"]
        )
        unlabel_data_k = self.add_label(
            unlabel_data_k, joint_proposal_dict["proposals_pseudo_roih"]
        )

        all_label_data = label_data_q + label_data_k
        global all_unlabel_data
        all_unlabel_data = unlabel_data_q

        all_label_data = change_classes(all_label_data)
        all_unlabel_data = change_classes(all_unlabel_data)

        # import pprint
        # print('for this it works')
        # pprint.pprint(all_label_data)
        # print('not so much here')
        # pprint.pprint(all_unlabel_data)

        all_unlabel_data = filter_instances(all_unlabel_data)
        
        try:
            record_all_label_data, _, _, _ = self.model(all_label_data, branch='supervised')
        except FloatingPointError:
            import pprint
            print('all label data')
            pprint.pprint(all_label_data)

            print('nancheck')
            for data in all_label_data:
                print(torch.isnan(data['image']).sum())

            dummy = jfoisndvoc

        record_dict.update(record_all_label_data)

        record_all_unlabel_data, _, _, _ = self.model(
            all_unlabel_data, branch="supervised"
        )
        new_record_all_unlabel_data = {}
        for key in record_all_unlabel_data.keys():
            new_record_all_unlabel_data[key + "_pseudo"] = record_all_unlabel_data[
                key
            ]
        record_dict.update(new_record_all_unlabel_data)

        # print('--------- current state of progression ----------')
        # print('final record dict before adjusting losses')
        # print(record_dict)

        # weight losses
        loss_dict = {}
        for key in record_dict.keys():
            if key[:4] == "loss":
                if key == "loss_rpn_loc_pseudo" or key == "loss_box_reg_pseudo":
                    # pseudo bbox regression <- 0
                    loss_dict[key] = record_dict[key] * 0
                elif key[-6:] == "pseudo":  # unsupervised loss
                    loss_dict[key] = (
                        record_dict[key] *
                        self.cfg.SEMISUPNET.UNSUP_LOSS_WEIGHT
                    )
                else:  # supervised loss
                    loss_dict[key] = record_dict[key] * 1

        losses = sum(loss_dict.values())

        metrics_dict = record_dict
        metrics_dict["data_time"] = data_time
        self._write_metrics(metrics_dict)

        self.optimizer.zero_grad()
        losses.backward()

        self.optimizer.step()




        # print('step done')



    
    @classmethod
    def build_train_loader(cls, cfg):
        mapper = MaxarDatasetMapper(cfg, True)
        # return build_detection_semisup_train_loader(cfg, mapper=None)
        return build_maxar_loader_semisup_two_crops(cfg, mapper)
            



In [110]:
args = default_argument_parser().parse_args()
args.config_file = 'configs/coco_supervision/faster_rcnn_R_50_FPN_sup2_run1.yaml'
cfg = setup(args)
cfg.defrost()
cfg.SEMISUPNET.Trainer = 'ubteacher'
cfg.SOLVER.IMG_PER_BATCH_LABEL = 1
cfg.SOLVER.IMG_PER_BATCH_UNLABEL = 1
cfg.INPUT.FORMAT = 'RGB'
cfg.DATALOADER.ASPECT_RATIO_GROUPING = True
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS = False
cfg.INPUT.MIN_SIZE_TRAIN = (256, 512, 640, 672, 704, 736, 768, 800)
cfg.SEMISUPNET.BURN_UP_STEP = 0
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.STEPS = [100, 200]
cfg.MODEL.META_ARCHITECTURE = 'MaxarPseudoLabGeneralizedRCNN'
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
model_path = os.path.join('/content', 'drive', 'MyDrive', 'PyPSA_Africa_images', 'models', 
                            '2021-11-29_frcnn_180000_dukeset', 'model_final.pth')

cfg.MODEL.WEIGHTS = model_path

cfg.MODEL.PROPOSAL_GENERATOR.RPN = 'PseudoLabRPN'
cfg.MODEL.ROI_HEADS.NAME = 'MaxarROIHeads'
cfg.freeze()

trainer = MaxarUBTeacher(cfg)
trainer.train()



[02/20 16:58:11 detectron2]: Rank of current process: 0. World size: 1
[02/20 16:58:11 detectron2]: Environment info:
----------------------  ----------------------------------------------------------------
sys.platform            linux
Python                  3.7.12 (default, Jan 15 2022, 18:48:18) [GCC 7.5.0]
numpy                   1.21.5
detectron2              0.6 @/usr/local/lib/python3.7/dist-packages/detectron2
Compiler                GCC 7.3
CUDA compiler           CUDA 11.1
detectron2 arch flags   3.7, 5.0, 5.2, 6.0, 6.1, 7.0, 7.5, 8.0, 8.6
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.10.0+cu111 @/usr/local/lib/python3.7/dist-packages/torch
PyTorch debug build     False
GPU available           Yes
GPU 0                   Tesla P100-PCIE-16GB (arch=6.0)
Driver version          460.32.03
CUDA_HOME               /usr/local/cuda
Pillow                  7.1.2
torchvision             0.11.1+cu111 @/usr/local/lib/python3.7/dist-packages/torchvision
torchvision arch fl

/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride


[02/20 16:58:22 d2.utils.events]:  eta: 0:05:37  iter: 19  total_loss: 21.48  loss_cls: 5.371  loss_box_reg: 0.7865  loss_rpn_cls: 1.303  loss_rpn_loc: 0.3951  loss_cls_pseudo: 2.596  loss_box_reg_pseudo: 0.5517  loss_rpn_cls_pseudo: 3.497  loss_rpn_loc_pseudo: 3.097  time: 0.3503  data_time: 0.0591  lr: 1.171e-05  max_mem: 4414M
[02/20 16:58:29 d2.utils.events]:  eta: 0:05:21  iter: 39  total_loss: 6.15  loss_cls: 0.2702  loss_box_reg: 0.1414  loss_rpn_cls: 0.3898  loss_rpn_loc: 0.08547  loss_cls_pseudo: 1.391  loss_box_reg_pseudo: 0.7446  loss_rpn_cls_pseudo: 0.5159  loss_rpn_loc_pseudo: 1.995  time: 0.3409  data_time: 0.0067  lr: 1.351e-05  max_mem: 4470M
[02/20 16:58:36 d2.utils.events]:  eta: 0:05:30  iter: 59  total_loss: 5.392  loss_cls: 0.1674  loss_box_reg: 0.149  loss_rpn_cls: 0.303  loss_rpn_loc: 0.06721  loss_cls_pseudo: 1.02  loss_box_reg_pseudo: 0.9709  loss_rpn_cls_pseudo: 0.4657  loss_rpn_loc_pseudo: 2.061  time: 0.3542  data_time: 0.0077  lr: 1.531e-05  max_mem: 4486M


FileNotFoundError: ignored

In [103]:
checkpointer.__dict__

CfgNode({'LR_SCHEDULER_NAME': 'WarmupMultiStepLR', 'MAX_ITER': 1000, 'BASE_LR': 0.01, 'MOMENTUM': 0.9, 'NESTEROV': False, 'WEIGHT_DECAY': 0.0001, 'WEIGHT_DECAY_NORM': 0.0, 'GAMMA': 0.1, 'STEPS': (179990, 179995), 'WARMUP_FACTOR': 0.001, 'WARMUP_ITERS': 1000, 'WARMUP_METHOD': 'linear', 'CHECKPOINT_PERIOD': 5000, 'IMS_PER_BATCH': 16, 'REFERENCE_WORLD_SIZE': 0, 'BIAS_LR_FACTOR': 1.0, 'WEIGHT_DECAY_BIAS': None, 'CLIP_GRADIENTS': CfgNode({'ENABLED': False, 'CLIP_TYPE': 'value', 'CLIP_VALUE': 1.0, 'NORM_TYPE': 2.0}), 'AMP': CfgNode({'ENABLED': False}), 'IMG_PER_BATCH_LABEL': 1, 'IMG_PER_BATCH_UNLABEL': 1, 'FACTOR_LIST': (1,)})

### Our model methods

In [47]:
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
import helper
import numpy as np
import os
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.modeling import build_model
from detectron2.data.build import build_batch_data_loader
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from torchvision import transforms


def get_true_images(dir, num, show=False):
    '''
    loads a list of images that all have a tower in them. dir has to be a directory
    only containing such images

    Parameters
    ----------
    dir : str
        directory of true examples
    num : int
        length of list returned
    show : bool
        if True, some of the examples are shown

    Returns
    ----------
    imgs : list of 3 x height x width np.ndarray
        list of obtained images
    '''

    imgs = os.listdir(dir) 
    imgs = [os.path.join(dir, img) for img in imgs]
    # randomize order
    np.random.shuffle(imgs)

    imgs = [cv2.imread(img) for img in imgs[:num]]

    if show:
        print('here are 10 images we have obtained')
        for img in imgs[:10]:
            _, ax = plt.subplots(1, 1, figsize=(4, 4))
            ax.imshow(img)
            plt.show()
    
    return imgs    


def build_predictor_model(threshold=0.2, model_path=None):
    '''
    Returns predictor and model using detectron2 from files stored in the
    PyPSA Africa drive

    Parameters
    ----------
    threshold : float
        detection threshold
    model_path : str
        path to .pth file

    Returns
    ---------
    predictor : detectron2.DefaultPredictor
    model : torch.nn.Module

    ''' 

    print('Building predictor from path:')
    print(model_path)

    ds_path = f'/content/drive/My Drive/PyPSA_Africa_images/datasets/duke_train/data/' 
    json_path = f'/content/drive/My Drive/PyPSA_Africa_images/datasets/duke_train/labels.json'
    ds_name = 'duke'
    
    if ds_name in DatasetCatalog.list():
        DatasetCatalog.remove(ds_name)
        MetadataCatalog.remove(ds_name)
    
    register_coco_instances(ds_name, {}, json_path, ds_path)
    
    frcnn = 'faster_rcnn_R_101_FPN_3x.yaml'
    
    cfg = get_cfg() # Model Config
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/"+frcnn))

    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold
    if model_path is None:
        model_path = os.path.join('/content', 'drive', 'MyDrive', 'PyPSA_Africa_images', 'models', 
                                    '2021-11-29_frcnn_180000_dukeset', 'model_final.pth')
    
    cfg.MODEL.WEIGHTS = model_path

    print('working with path: ', model_path)
    cfg.INPUT.FORMAT = 'BGR'

    predictor = DefaultPredictor(cfg)                                
    model = build_model(cfg)

    print('standard frcnn cfg')
    print(cfg)

    global frcnn_cfg
    frcnn_cfg = cfg

    checkpointer = DetectionCheckpointer(model)
    checkpointer.load(model_path)

    # loader = build_batch_data_loader(cfg)

    return predictor, model#, loader


def eval_predictor(imgs, model_path=None, threshold=0.1):
    '''
    Method to check performance of model on some imgs (only chekc via plotting, does 
    not return precision scores)

    Parameters
    ----------
    imgs : list of np.array 
        images on which inference will run
    model_path : str
        path to model .pth file
    threshold : float
        cutoff for what is considered as an instance
    
    Returns
    ----------
    -

    '''

    print('Evaluating model stored under:')
    print(model_path)

    predictor, model = build_predictor_model(threshold=threshold, model_path=model_path)
    # model.eval()

    for img in imgs:

        out = predictor(img)
        # out = predictor(img[:,:,::-1])
        v = Visualizer(img, MetadataCatalog.get('duke'), scale=1.5)
        out = v.draw_instance_predictions(out["instances"].to("cpu"))
        cv2_imshow(out.get_image())

        with torch.no_grad():
            
            img = predictor.aug.get_transform(img).apply_image(img)
            img = torch.as_tensor(img.astype('float32').transpose(2, 0, 1))
            x = [{'image': img, 'width': 256, 'height': 256}]
            pred = predictor.model(x)
            print(pred)


'''
if __name__ == '__main__':
    
    duke_img_dir = '/content/drive/MyDrive/PyPSA_Africa_images/datasets/duke_val/data/'
    duke_imgs = get_true_images(duke_img_dir, 20)
    
    model_path = '/content/drive/MyDrive/PyPSA_Africa_images/notebooks/pypsa-africa/PISA_models_duke/model_final.pth'
    model_path_cycle = '/content/drive/MyDrive/PyPSA_Africa_images/PISA_models/11_01_2022_fake_maxar_train/model_final.pth'
    
    print('The cycle GAN trained model')
    eval_predictor(duke_imgs, model_path=model_path_cycle)
    
    print('The regular trained model')
    eval_predictor(duke_imgs, model_path=model_path)
'''

"\nif __name__ == '__main__':\n    \n    duke_img_dir = '/content/drive/MyDrive/PyPSA_Africa_images/datasets/duke_val/data/'\n    duke_imgs = get_true_images(duke_img_dir, 20)\n    \n    model_path = '/content/drive/MyDrive/PyPSA_Africa_images/notebooks/pypsa-africa/PISA_models_duke/model_final.pth'\n    model_path_cycle = '/content/drive/MyDrive/PyPSA_Africa_images/PISA_models/11_01_2022_fake_maxar_train/model_final.pth'\n    \n    print('The cycle GAN trained model')\n    eval_predictor(duke_imgs, model_path=model_path_cycle)\n    \n    print('The regular trained model')\n    eval_predictor(duke_imgs, model_path=model_path)\n"

In [49]:
predictor, model = build_predictor_model()

Building predictor from path:
None
working with path:  /content/drive/MyDrive/PyPSA_Africa_images/models/2021-11-29_frcnn_180000_dukeset/model_final.pth
[02/20 15:10:30 fvcore.common.checkpoint]: [Checkpointer] Loading from /content/drive/MyDrive/PyPSA_Africa_images/models/2021-11-29_frcnn_180000_dukeset/model_final.pth ...
standard frcnn cfg
CUDNN_BENCHMARK: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  FILTER_EMPTY_ANNOTATIONS: True
  NUM_WORKERS: 4
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: ()
  PROPOSAL_FILES_TRAIN: ()
  TEST: ('coco_2017_val',)
  TRAIN: ('coco_2017_train',)
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: False
    SIZE: [0.9, 0.9]
    TYPE: relative_range
  FORMAT: BGR
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (640, 672, 704, 736, 768, 800)
  MIN_SIZE_TRAIN_SAMPLING: choic

In [32]:
lq, lk, uq, uk = next(loader)

In [33]:
uk = trainer.remove_label(uk)

In [34]:
# _, prop_rpn, prop_roih, _ = teacher(uk, branch='unsup_data_weak')

In [46]:
import detectron2

with torch.no_grad():
    out = uk

    current = model 
    print(type(current))
    model.eval()

    images = current.preprocess_image(out)

    if "instances" in uk[0]:
        gt_instances = [x["instances"].to(teacher.device) for x in uk]
    else:
        gt_instances = None

    features = current.backbone(images.tensor)

    branch = 'unsup_data_weak'

    print(help(current.proposal_generator.forward))

    if branch == 'unsup_data_weak':

        # not using isistance as on type inherits the other
        if type(current) == detectron2.modeling.meta_arch.rcnn.GeneralizedRCNN:
            print('detected d2.GeneralizedRCNN')
            prop_rpn, _ = current.proposal_generator(images, features, None)
            prop_roih, ROI_PRED = current.roi_heads(
                images,
                features,
                prop_rpn,
            )

        elif type(current) == MaxarPseudoLabGeneralizedRCNN:  
            print('detected MaxarGeneralizedRCNN')
            prop_rpn, _ = current.proposal_generator(images, features, None, compute_loss=False)
            prop_roih, ROI_PRED = current.roi_heads(
                images,
                features,
                prop_rpn,
                targets=None,
                compute_loss=False,
                branch=branch
            )
        else:
            assert False, 'Unrecognized network type!'




    # pprint.pprint(features)
    print('results')
    pprint.pprint(prop_roih[0])
    print(type(prop_roih))
    
 


<class 'detectron2.modeling.meta_arch.rcnn.GeneralizedRCNN'>
Help on method forward in module detectron2.modeling.proposal_generator.rpn:

forward(images: detectron2.structures.image_list.ImageList, features: Dict[str, torch.Tensor], gt_instances: Union[List[detectron2.structures.instances.Instances], NoneType] = None) method of detectron2.modeling.proposal_generator.rpn.RPN instance
    Args:
        images (ImageList): input images of length `N`
        features (dict[str, Tensor]): input data as a mapping from feature
            map name to tensor. Axis 0 represents the number of images `N` in
            the input data; axes 1-3 are channels, height, and width, which may
            vary between feature maps (e.g., if a feature pyramid is used).
        gt_instances (list[Instances], optional): a length `N` list of `Instances`s.
            Each `Instances` stores ground-truth instances for the corresponding image.
    
    Returns:
        proposals: list[Instances]: contains fie

/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride


In [14]:
type(student)

ubteacher.modeling.meta_arch.rcnn.TwoStagePseudoLabGeneralizedRCNN

In [18]:
!pip install jax

In [77]:
mod = 'roi_heads'
print(type(getattr(teacher, mod)))
print(type(getattr(model, mod)))
print(cfg.MODEL.WEIGHTS)
print(frcnn_cfg.MODEL.WEIGHTS)

<class 'ubteacher.modeling.roi_heads.roi_heads.StandardROIHeadsPseudoLab'>
<class 'detectron2.modeling.roi_heads.roi_heads.StandardROIHeads'>
detectron2://ImageNetPretrained/MSRA/R-50.pkl
/content/drive/MyDrive/PyPSA_Africa_images/models/2021-11-29_frcnn_180000_dukeset/model_final.pth


In [81]:
frcnn = 'faster_rcnn_R_101_FPN_3x.yaml'

test_cfg = get_cfg() # Model Config
test_cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/"+frcnn))

test_cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
model_path = os.path.join('/content', 'drive', 'MyDrive', 'PyPSA_Africa_images', 'models', 
                            '2021-11-29_frcnn_180000_dukeset', 'model_final.pth')

test_cfg.MODEL.WEIGHTS = model_path

test_cfg.MODEL.META_ARCHITECTURE = 'MaxarPseudoLabGeneralizedRCNN'
test_cfg.MODEL.PROPOSAL_GENERATOR.RPN = 'PseudoLabRPN'
test_cfg.MODEL.ROI_HEADS.NAME = 'MaxarROIHeads'

test_model = build_model(test_cfg)




In [80]:
import torch
from typing import Dict, List, Optional, Tuple, Union
from detectron2.structures import Boxes, ImageList, Instances, pairwise_iou
from detectron2.modeling.proposal_generator.proposal_utils import (
    add_ground_truth_to_proposals,
)
from detectron2.utils.events import get_event_storage
from detectron2.modeling.roi_heads.box_head import build_box_head
from detectron2.layers import ShapeSpec
from detectron2.modeling.roi_heads import (
    ROI_HEADS_REGISTRY,
    StandardROIHeads,
)
from detectron2.modeling.roi_heads.fast_rcnn import FastRCNNOutputLayers

import numpy as np
from detectron2.modeling.poolers import ROIPooler

if 'MaxarROIHeads' in ROI_HEADS_REGISTRY:
    ROI_HEADS_REGISTRY.__dict__['_obj_map'].pop('MaxarROIHeads')

@ROI_HEADS_REGISTRY.register()
class MaxarROIHeads(StandardROIHeads):
    @classmethod
    def _init_box_head(cls, cfg, input_shape):
        # fmt: off
        in_features       = cfg.MODEL.ROI_HEADS.IN_FEATURES
        pooler_resolution = cfg.MODEL.ROI_BOX_HEAD.POOLER_RESOLUTION
        pooler_scales     = tuple(1.0 / input_shape[k].stride for k in in_features)
        sampling_ratio    = cfg.MODEL.ROI_BOX_HEAD.POOLER_SAMPLING_RATIO
        pooler_type       = cfg.MODEL.ROI_BOX_HEAD.POOLER_TYPE
        # fmt: on

        in_channels = [input_shape[f].channels for f in in_features]
        # Check all channel counts are equal
        assert len(set(in_channels)) == 1, in_channels
        in_channels = in_channels[0]

        box_pooler = ROIPooler(
            output_size=pooler_resolution,
            scales=pooler_scales,
            sampling_ratio=sampling_ratio,
            pooler_type=pooler_type,
        )
        box_head = build_box_head(
            cfg,
            ShapeSpec(
                channels=in_channels, height=pooler_resolution, width=pooler_resolution
            ),
        )

        box_predictor = FastRCNNOutputLayers(cfg, box_head.output_shape)    
        return {
            "box_in_features": in_features,
            "box_pooler": box_pooler,
            "box_head": box_head,
            "box_predictor": box_predictor,
        }


    def forward(
        self,
        images: ImageList,
        features: Dict[str, torch.Tensor],
        proposals: List[Instances],
        targets: Optional[List[Instances]] = None,
        compute_loss=True,
        branch="",
        compute_val_loss=False,
    ) -> Tuple[List[Instances], Dict[str, torch.Tensor]]:

        del images
        if self.training and compute_loss:  # apply if training loss
            assert targets
            # 1000 --> 512
            proposals = self.label_and_sample_proposals(
                proposals, targets, branch=branch
            )
        elif compute_val_loss:  # apply if val loss
            assert targets
            # 1000 --> 512
            temp_proposal_append_gt = self.proposal_append_gt
            self.proposal_append_gt = False
            proposals = self.label_and_sample_proposals(
                proposals, targets, branch=branch
            )  # do not apply target on proposals
            self.proposal_append_gt = temp_proposal_append_gt
        del targets

        if (self.training and compute_loss) or compute_val_loss:
            losses, _ = self._forward_box(
                features, proposals, compute_loss, compute_val_loss, branch
            )
            return proposals, losses
        else:
            pred_instances, predictions = self._forward_box(
                features, proposals, compute_loss, compute_val_loss, branch
            )

            return pred_instances, predictions


    def _forward_box(
        self,
        features: Dict[str, torch.Tensor],
        proposals: List[Instances],
        compute_loss: bool = True,
        compute_val_loss: bool = False,
        branch: str = "",
    ) -> Union[Dict[str, torch.Tensor], List[Instances]]:
        features = [features[f] for f in self.box_in_features]
        box_features = self.box_pooler(features, [x.proposal_boxes for x in proposals])
        box_features = self.box_head(box_features)
        predictions = self.box_predictor(box_features)
        del box_features

        if (
            self.training and compute_loss
        ) or compute_val_loss:  # apply if training loss or val loss
            losses = self.box_predictor.losses(predictions, proposals)

            if self.train_on_pred_boxes:
                with torch.no_grad():
                    pred_boxes = self.box_predictor.predict_boxes_for_gt_classes(
                        predictions, proposals
                    )
                    for proposals_per_image, pred_boxes_per_image in zip(
                        proposals, pred_boxes
                    ):
                        proposals_per_image.proposal_boxes = Boxes(pred_boxes_per_image)
            return losses, predictions
        else:

            pred_instances, _ = self.box_predictor.inference(predictions, proposals)
            return pred_instances, predictions


    @torch.no_grad()
    def label_and_sample_proposals(
        self, proposals: List[Instances], targets: List[Instances], branch: str = ""
    ) -> List[Instances]:
        gt_boxes = [x.gt_boxes for x in targets]
        if self.proposal_append_gt:
            proposals = add_ground_truth_to_proposals(gt_boxes, proposals)

        proposals_with_gt = []

        num_fg_samples = []
        num_bg_samples = []
        for proposals_per_image, targets_per_image in zip(proposals, targets):
            has_gt = len(targets_per_image) > 0
            match_quality_matrix = pairwise_iou(
                targets_per_image.gt_boxes, proposals_per_image.proposal_boxes
            )
            matched_idxs, matched_labels = self.proposal_matcher(match_quality_matrix)
            sampled_idxs, gt_classes = self._sample_proposals(
                matched_idxs, matched_labels, targets_per_image.gt_classes
            )

            proposals_per_image = proposals_per_image[sampled_idxs]
            proposals_per_image.gt_classes = gt_classes

            if has_gt:
                sampled_targets = matched_idxs[sampled_idxs]
                for (trg_name, trg_value) in targets_per_image.get_fields().items():
                    if trg_name.startswith("gt_") and not proposals_per_image.has(
                        trg_name
                    ):
                        proposals_per_image.set(trg_name, trg_value[sampled_targets])
            else:
                gt_boxes = Boxes(
                    targets_per_image.gt_boxes.tensor.new_zeros((len(sampled_idxs), 4))
                )
                proposals_per_image.gt_boxes = gt_boxes

            num_bg_samples.append((gt_classes == self.num_classes).sum().item())
            num_fg_samples.append(gt_classes.numel() - num_bg_samples[-1])
            proposals_with_gt.append(proposals_per_image)

        storage = get_event_storage()
        storage.put_scalar(
            "roi_head/num_target_fg_samples_" + branch, np.mean(num_fg_samples)
        )
        storage.put_scalar(
            "roi_head/num_target_bg_samples_" + branch, np.mean(num_bg_samples)
        )

        return proposals_with_gt